In [1]:
import os
import shutil

import pandas as pd
import utils
from docx import Document
from docx.shared import Pt

# Notebook 04: Generate TEOTIL3 Word report

Inserts maps and tables into Word using the structure required for the annual TEOTIL report. Text can then be added/edited afterwards.

In [2]:
# Output file. A new file will be created named
# "../report_data/{year}/teotil_report_{year}_draft{draft_no}.docx"
# NOTE: If this already exists, it will be overwritten!
year = 2023
draft_no = 1

# Template version to use
template_version = "2025-04-24"

In [3]:
# Dict mapping headings in the Word template to tables/images that should be inserted below them
# If the value is None, the file path can be constructed from the heading alone
data_dict = {
    "Kart over kildepunkter": [
        f"../report_data/{year}/plots/large-wastewater.png",
        f"../report_data/{year}/plots/industry.png",
        f"../report_data/{year}/plots/aquaculture.png",
    ],
    # Definerte kystavsnitt
    "Norges kystområder: fosfor (tonn)": None,
    "Norges kystområder: nitrogen (tonn)": None,
    "Norges kystområder: karbon (tonn)": None,
    "Sverige – Strømtangen fyr (Fredrikstad): fosfor (tonn)": None,
    "Sverige – Strømtangen fyr (Fredrikstad): nitrogen (tonn)": None,
    "Sverige – Strømtangen fyr (Fredrikstad): karbon (tonn)": None,
    "Indre Oslofjord (nord for Drøbak): fosfor (tonn)": None,
    "Indre Oslofjord (nord for Drøbak): nitrogen (tonn)": None,
    "Indre Oslofjord (nord for Drøbak): karbon (tonn)": None,
    "Hele Oslofjord (Svenskegrensa - Kragerø): fosfor (tonn)": None,
    "Hele Oslofjord (Svenskegrensa - Kragerø): nitrogen (tonn)": None,
    "Hele Oslofjord (Svenskegrensa - Kragerø): karbon (tonn)": None,
    "Svenskegrensa – Lindesnes: fosfor (tonn)": None,
    "Svenskegrensa – Lindesnes: nitrogen (tonn)": None,
    "Svenskegrensa – Lindesnes: karbon (tonn)": None,
    "Lindesnes – Stad: fosfor (tonn)": None,
    "Lindesnes – Stad: nitrogen (tonn)": None,
    "Lindesnes – Stad: karbon (tonn)": None,
    "Stad – Russland: fosfor (tonn)": None,
    "Stad – Russland: nitrogen (tonn)": None,
    "Stad – Russland: karbon (tonn)": None,
    # Norske vannregioner
    "Kartpresentajon av tilførsler av næringssalter til sjø fra vannregionene": [
        f"../report_data/{year}/plots/vannregion_p.png",
        f"../report_data/{year}/plots/vannregion_n.png",
        f"../report_data/{year}/plots/vannregion_c.png",
    ],
    "Glomma: fosfor (tonn)": None,
    "Glomma: nitrogen (tonn)": None,
    "Glomma: karbon (tonn)": None,
    "Vest-Viken: fosfor (tonn)": None,
    "Vest-Viken: nitrogen (tonn)": None,
    "Vest-Viken: karbon (tonn)": None,
    "Agder: fosfor (tonn)": None,
    "Agder: nitrogen (tonn)": None,
    "Agder: karbon (tonn)": None,
    "Rogaland: fosfor (tonn)": None,
    "Rogaland: nitrogen (tonn)": None,
    "Rogaland: karbon (tonn)": None,
    "Vestland: fosfor (tonn)": None,
    "Vestland: nitrogen (tonn)": None,
    "Vestland: karbon (tonn)": None,
    "Møre og Romsdal: fosfor (tonn)": None,
    "Møre og Romsdal: nitrogen (tonn)": None,
    "Møre og Romsdal: karbon (tonn)": None,
    "Trøndelag: fosfor (tonn)": None,
    "Trøndelag: nitrogen (tonn)": None,
    "Trøndelag: karbon (tonn)": None,
    "Nordland: fosfor (tonn)": None,
    "Nordland: nitrogen (tonn)": None,
    "Nordland: karbon (tonn)": None,
    "Troms: fosfor (tonn)": None,
    "Troms: nitrogen (tonn)": None,
    "Troms: karbon (tonn)": None,
    "Finnmark: fosfor (tonn)": None,
    "Finnmark: nitrogen (tonn)": None,
    "Finnmark: karbon (tonn)": None,
    # Norske forvaltingsplanområder
    "Kartpresentasjon av næringssalttilførsler til forvaltningsplanområder": [
        f"../report_data/{year}/plots/forvaltom_p.png",
        f"../report_data/{year}/plots/forvaltom_n.png",
        f"../report_data/{year}/plots/forvaltom_c.png",
    ],
    "Nordsjøen: fosfor (tonn)": None,
    "Nordsjøen: nitrogen (tonn)": None,
    "Nordsjøen: karbon (tonn)": None,
    "Norskehavet: fosfor (tonn)": None,
    "Norskehavet: nitrogen (tonn)": None,
    "Norskehavet: karbon (tonn)": None,
    "Barentshavet: fosfor (tonn)": None,
    "Barentshavet: nitrogen (tonn)": None,
    "Barentshavet: karbon (tonn)": None,
}

In [4]:
# Paths to report data (from notebooks 01 and 02)
data_fold = f"../report_data/{year}/tables"
plot_fold = f"../report_data/{year}/plots"

# Copy template. NOTE: This will overwrite 'docx_path' if it already exists!
docx_path = f"../report_data/{year}/teotil_report_{year}_draft{draft_no}.docx"
template_path = (
    f"../report_templates/teotil3_report_blank_template_v{template_version}.docx"
)
shutil.copyfile(template_path, docx_path)

# Open the document
doc = Document(docx_path)

# Set styles for 'Normal' template in this doc
style = doc.styles["Normal"]
font = style.font
font.name = "Calibri"
font.size = Pt(11)
p_format = style.paragraph_format
p_format.space_before = Pt(0)
p_format.space_after = Pt(0)

# Loop over headings in template
for para in doc.paragraphs:
    heading = para.text
    if heading in data_dict.keys():
        print(f"Processing: {heading}")

        if data_dict[heading]:
            # Processing maps
            # Need to process files in reverse order as each element will be
            # inserted immediately under the header text i.e. start with last first
            flist = reversed(data_dict[heading])
            for fpath in flist:
                new_para = utils.insert_para_after(para)
                utils.insert_image(new_para, fpath)
        else:
            # Processing tables and time series plots
            fname = utils.filename_from_heading(heading)
            csv_path = os.path.join(data_fold, f"{fname}.csv")
            plot_path = os.path.join(plot_fold, f"{fname}.png")
            df = pd.read_csv(csv_path).rename(
                columns={"Akvakultur": "Akvakult.", "Menneskeskapt": "Menneske."}
            )
            new_para = utils.insert_para_after(para)
            new_para2 = utils.insert_para_after(new_para)
            new_para3 = utils.insert_para_after(new_para2)
            utils.insert_image(new_para3, plot_path)
            utils.insert_table(doc, new_para, df)

doc.save(docx_path)

print("Done.")

Processing: Kart over kildepunkter
Processing: Norges kystområder: fosfor (tonn)
Processing: Norges kystområder: nitrogen (tonn)
Processing: Norges kystområder: karbon (tonn)
Processing: Sverige – Strømtangen fyr (Fredrikstad): fosfor (tonn)
Processing: Sverige – Strømtangen fyr (Fredrikstad): nitrogen (tonn)
Processing: Sverige – Strømtangen fyr (Fredrikstad): karbon (tonn)
Processing: Indre Oslofjord (nord for Drøbak): fosfor (tonn)
Processing: Indre Oslofjord (nord for Drøbak): nitrogen (tonn)
Processing: Indre Oslofjord (nord for Drøbak): karbon (tonn)
Processing: Hele Oslofjord (Svenskegrensa - Kragerø): fosfor (tonn)
Processing: Hele Oslofjord (Svenskegrensa - Kragerø): nitrogen (tonn)
Processing: Hele Oslofjord (Svenskegrensa - Kragerø): karbon (tonn)
Processing: Svenskegrensa – Lindesnes: fosfor (tonn)
Processing: Svenskegrensa – Lindesnes: nitrogen (tonn)
Processing: Svenskegrensa – Lindesnes: karbon (tonn)
Processing: Lindesnes – Stad: fosfor (tonn)
Processing: Lindesnes – St